In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from utils import data_generator

IMAGE_FOLDER = "debug_images"


image_paths = [IMAGE_FOLDER + "/" + name for name in next(os.walk(IMAGE_FOLDER))[2]]
gen = data_generator(image_paths)

In [ ]:
# LAUNCH THIS BLOCK TO TRY OUT THE DATA GENERATOR
feature, labels = next(gen)
plt.imshow(feature, cmap="gray")

In [ ]:
# LAUNCH THIS BLOCK TO VISUALIZE THE PRE PROCESS
from utils import pre_process, process_output

def plot_visualization(image):
    """
    Function to visualize the best output we could have from
    a perfect a model, given the information lost in the pre-process.
    (especially the resizing makes us lose a lot of details)
    """
    features, labels = pre_process(image)
    output = process_output(features, labels, np.shape(image))
    
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("original image")
    
    plt.subplot(1, 3, 2)
    plt.imshow(output)
    plt.title("optimal network output")
    
    plt.subplot(1, 3, 3)
    plt.imshow(features, cmap="gray")
    plt.title("network input")
    
    
list_names = np.random.choice(next(os.walk(IMAGE_FOLDER))[2], 6, replace=False)
for name in list_names:
    bgr_image = cv2.imread(IMAGE_FOLDER + "/" + name)
    rgb_image = bgr_image[:, :, ::-1]
    
    plt.figure(figsize=(15, 5))
    plot_visualization(rgb_image)